# KBO 1982~2020 타자 기록 크롤링(statiz.co.kr)

## Chrome driver 설치

보편적으로 사용하는 Chrome 웹 브라우저를 이용한 크롤링입니다.

본인이 사용중인 크롬 버전과 맞는 driver를 설치해야 소스코드를 실행할 수 있습니다.

먼저 아래 링크에서 현재 사용중인 크롬 버전을 확인합니다.

[크롬 버전 확인](chrome://version)

가장 첫 줄에 적혀있는 게 크롬 버전입니다.

아래 링크에서 버전과 맞는 드라이버를 설치하면 됩니다.

(크롬을 이참에 업데이트 해주는 것도 좋겠습니다.)

[크롬 드라이버 다운로드](https://sites.google.com/a/chromium.org/chromedriver/downloads)

다운로드하면 압축파일이 하나 있는데, 압축을 해제한 Chrome driver 파일을

이 파일의 경로에 놓으면 소스코드를 그대로 실행하셔도 무방합니다.

## 크롤링

In [1]:
# 필요한 라이브러리 import
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# setup Driver | Chrome: 크롬 드라이버를 사용하는 driver 생성
driver = webdriver.Chrome('./chromedriver')  # Chrome driver 경로
driver.implicitly_wait(10)

result = []
position = ['P', 'C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF', 'DH', 'PH', 'PR']
rows = 400  # 한 번에 가져올 row 수
reps = 1  # 반복

for i in range(1982, 2021):  # 시작년도, 종료년도 + 1
    count = 0
    for j in range(reps):
        # url 접근
        driver.get('http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=' + str(i) + '&ye=' + str(i) + '&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=' + str(rows) + '&pa=' + str(j * rows) + '&si=&cn=')  # sn: data row 수, pn: 이전 페이지의 마지막 row 번호
        
        # 데이터 완전히 읽도록 대기 (데이터 유실되면 sleep을 늘려주거나, rows를 줄이고 rep를 늘려주세요.)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tr = soup.select('table#mytable.table.table-striped.table-responsive.table-condensed.no-space.table-bordered > tbody > tr')
        if(len(tr) == 0):  # 선수 데이터 row 수가 0인 경우 조기 종료
            break
        # Row 수 계산
        for r in tr:
            row = []
            td = r.find_all('td')
            for d in td:
                if d == td[2]:  # 팀(시즌 + 팀 + 포지션)
                    span = d.find_all('span')
                    year, team, pos, teamTag = span[1].text.strip(), '', '', ''

                    if int(year) > 81: year = '19' + year
                    else: year = '20' + year

                    if span[-1].text.strip() in position:
                        teamTag = span[2:-1]                                
                        pos = span[-1].text.strip()
                    else:
                        teamTag = span[2:]

                    for t in teamTag:
                        team += t.text.strip()

                    row.append(year)  # 시즌
                    row.append(team)  # 팀(시즌 + 팀 + 포지션)
                    row.append(pos)   # 포지션
                else:
                    row.append(d.text.strip())
            if(len(row) > 0):
                count += 1  # Row 수 세기
                result.append(row)
    # Row 수 출력
    print(i, '년: ', count, sep='')

data = pd.DataFrame(result)

1982년: 108
1983년: 117
1984년: 133
1985년: 144
1986년: 172
1987년: 167
1988년: 178
1989년: 190
1990년: 198
1991년: 235
1992년: 235
1993년: 217
1994년: 233
1995년: 227
1996년: 239
1997년: 232
1998년: 202
1999년: 198
2000년: 202
2001년: 214
2002년: 215
2003년: 216
2004년: 224
2005년: 206
2006년: 193
2007년: 277
2008년: 278
2009년: 278
2010년: 269
2011년: 263
2012년: 284
2013년: 257
2014년: 306
2015년: 381
2016년: 378
2017년: 360
2018년: 332
2019년: 357
2020년: 262


## 불필요한 데이터 제거 및 csv 저장

In [2]:
# 컬럼 확인
data.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,1,백인천,1982,M,DH,5.88,72,298,250,55,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,
1,2,윤동균,1982,O,LF,4.13,77,334,284,54,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,
2,3,이만수,1982,삼,C,4.12,78,322,270,46,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,
3,4,김봉연,1982,해,DH,3.68,74,304,269,55,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,
4,5,김우열,1982,O,CF,3.46,62,255,210,43,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,


In [3]:
# 중복된 column(WAR) 제거
data = data.drop(data.columns[5], axis='columns')

In [4]:
# 한글 column
data.columns = ['순서', '이름', '시즌', '팀', '포지션', '경기 수', '타석', '타수', '득점', '안타', '2루타', '3루타', '홈런', '총 루타', '타점', '도루', '도루실패', '볼넷', '사구', '고의사구', '삼진', '병살', '희생타', '희생플라이', '타율', '출루율', '장타율', 'OPS', 'wOBA', 'wRC+', 'WAR', 'WPA']
data.to_csv('KBO_1982_2020_Batter_KR.csv', encoding='cp949', index=False)
data.head()

,순서,이름,시즌,팀,포지션,경기 수,타석,타수,득점,안타,...,희생타,희생플라이,타율,출루율,장타율,OPS,wOBA,wRC+,WAR,WPA
0,1,백인천,1982,M,DH,72,298,250,55,103,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,
1,2,윤동균,1982,O,LF,77,334,284,54,97,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,
2,3,이만수,1982,삼,C,78,322,270,46,78,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,
3,4,김봉연,1982,해,DH,74,304,269,55,89,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,
4,5,김우열,1982,O,CF,62,255,210,43,65,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,


In [5]:
# 영어 column
data.columns = ['No', 'Name', 'Year', 'Team', 'Position', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'IBB', 'SO', 'GDP', 'SAC', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'wOBA', 'wRC+', 'WAR', 'WPA']
data.to_csv('KBO_1982_2020_Batter_EN.csv', encoding='cp949', index=False)
data.head()

,No,Name,Year,Team,Position,G,PA,AB,R,H,...,SAC,SF,AVG,OBP,SLG,OPS,wOBA,wRC+,WAR,WPA
0,1,백인천,1982,M,DH,72,298,250,55,103,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,
1,2,윤동균,1982,O,LF,77,334,284,54,97,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,
2,3,이만수,1982,삼,C,78,322,270,46,78,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,
3,4,김봉연,1982,해,DH,74,304,269,55,89,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,
4,5,김우열,1982,O,CF,62,255,210,43,65,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,


---------------------

# 연봉 크롤링

### 97년 ~

In [6]:
# 필요한 라이브러리 import
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# setup Driver | Chrome: 크롬 드라이버를 사용하는 driver 생성
driver = webdriver.Chrome('./chromedriver')  # Chrome driver 경로
driver.implicitly_wait(10)

result = []
for y in range(1997, 2021):
    count = 0
    t = 0
    while(t < 102):
        t += 1
        if t == 11: t = 101
            
        # url 접근
        driver.get('http://www.statiz.co.kr/salary.php?opt=0&sopt=' + str(y) + '&cnv=&pos=&te=' + str(t))
        time.sleep(1)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tr = soup.select('table.table.table-striped > tbody > tr')
        
        for r in tr:
            row = []
            td = r.find_all('td')
            
            if len(td) < 1 or td[3].text.strip() == "900,000":
                continue
                        
            for d in td:
                row.append(d.text.strip())
                
            if len(row) > 0:
                count += 1
                result.append(row)
                
    print(y, '년: ', count, sep='')
                
salary = pd.DataFrame(result)

1997년: 368
1998년: 371
1999년: 358
2000년: 351
2001년: 364
2002년: 359
2003년: 376
2004년: 382
2005년: 380
2006년: 371
2007년: 393
2008년: 416
2009년: 397
2010년: 397
2011년: 406
2012년: 425
2013년: 471
2014년: 543
2015년: 567
2016년: 656
2017년: 560
2018년: 509
2019년: 0
2020년: 681


In [7]:
salary.head()

,0,1,2,3,4
0,이강철,1997,해태,"11,000",4.247
1,김정수,1997,해태,"9,600",1.364
2,홍현우,1997,해태,"9,600",5.778
3,이대진,1997,해태,"9,000",4.891
4,이호성,1997,해태,"7,000",3.295


In [8]:
    # 사용하지 않는 column(WAR) 제거
    salary = salary.drop(salary.columns[4], axis='columns')

In [9]:
# 한글 column
salary.columns = ['이름', '시즌', '팀', '연봉']
salary.to_csv('KBO_1997_2020_Salary_KR.csv', encoding='cp949', index=False)
salary.head()

,이름,시즌,팀,연봉
0,이강철,1997,해태,"11,000"
1,김정수,1997,해태,"9,600"
2,홍현우,1997,해태,"9,600"
3,이대진,1997,해태,"9,000"
4,이호성,1997,해태,"7,000"


In [10]:
# 영어 column
salary.columns = ['Name', 'Year', 'Team', 'Salary']
salary.to_csv('KBO_1997_2020_Salary_EN.csv', encoding='cp949', index=False)
salary.head()

,Name,Year,Team,Salary
0,이강철,1997,해태,"11,000"
1,김정수,1997,해태,"9,600"
2,홍현우,1997,해태,"9,600"
3,이대진,1997,해태,"9,000"
4,이호성,1997,해태,"7,000"


---------------------

# 선수 기록 + 연봉

1. 공통된 컬럼 만들기(선수명 + 시즌 + 팀명 첫글자)
2. pandas merge

In [11]:
# 공통된 컬럼 만들기
data_for_merge = data.copy()
data_for_merge["for_merge"] = data["Name"] + data["Year"] + data["Team"]
data_for_merge.head()

,No,Name,Year,Team,Position,G,PA,AB,R,H,...,SF,AVG,OBP,SLG,OPS,wOBA,wRC+,WAR,WPA,for_merge
0,1,백인천,1982,M,DH,72,298,250,55,103,...,3,.412,.497,.740,1.237,.528,227.0,5.88,,백인천1982M
1,2,윤동균,1982,O,LF,77,334,284,54,97,...,4,.342,.428,.532,.960,.430,164.6,4.13,,윤동균1982O
2,3,이만수,1982,삼,C,78,322,270,46,78,...,1,.289,.399,.500,.899,.405,147.6,4.12,,이만수1982삼
3,4,김봉연,1982,해,DH,74,304,269,55,89,...,1,.331,.405,.636,1.040,.453,180.5,3.68,,김봉연1982해
4,5,김우열,1982,O,CF,62,255,210,43,65,...,1,.310,.428,.533,.961,.426,161.9,3.46,,김우열1982O


In [12]:
salary_for_merge = salary.copy()
salary_for_merge["for_merge"] = salary_for_merge["Name"] + salary_for_merge["Year"] + salary_for_merge["Team"].str.get(i=0)
salary_for_merge.head()

,Name,Year,Team,Salary,for_merge
0,이강철,1997,해태,"11,000",이강철1997해
1,김정수,1997,해태,"9,600",김정수1997해
2,홍현우,1997,해태,"9,600",홍현우1997해
3,이대진,1997,해태,"9,000",이대진1997해
4,이호성,1997,해태,"7,000",이호성1997해


In [13]:
salary_for_merge = salary_for_merge.drop(salary.columns[0:3], axis='columns')
salary_for_merge.head()

,Salary,for_merge
0,"11,000",이강철1997해
1,"9,600",김정수1997해
2,"9,600",홍현우1997해
3,"9,000",이대진1997해
4,"7,000",이호성1997해


In [14]:
# Merge
data_salary = data_for_merge.merge(salary_for_merge)
data_salary.head()

# 사용하지 않는 컬럼 제거
data_salary = data_salary.drop(data_salary.columns[-2], axis='columns')
data_salary.to_csv('KBO_1997_2020_Data_Salary_EN.csv', encoding='cp949', index=False)
data_salary.head()

,No,Name,Year,Team,Position,G,PA,AB,R,H,...,SF,AVG,OBP,SLG,OPS,wOBA,wRC+,WAR,WPA,Salary
0,2,양준혁,1997,삼,RF,126,561,442,94,145,...,8,.328,.455,.627,1.082,.457,189.3,8.17,,"12,000"
1,3,김기태,1997,쌍,1B,111,478,390,95,134,...,2,.344,.460,.636,1.096,.472,202.6,7.30,,"10,100"
2,4,최익성,1997,삼,CF,122,568,480,107,142,...,2,.296,.394,.494,.888,.395,148.0,7.10,,"3,000"
3,5,이승엽,1997,삼,1B,126,577,517,96,170,...,4,.329,.391,.598,.988,.432,172.3,6.78,,"6,500"
4,6,박재홍,1997,현,CF,96,403,331,77,108,...,2,.326,.442,.619,1.061,.458,193.7,6.65,,"5,000"


-------------------------

# 다음 해 연봉 추가

In [80]:
data_salary_next = data_salary.copy()

# 이름, 연도 오름차순 정렬
data_salary_next.sort_values(by=['Name', 'Year'], inplace=True)
data_salary_next = data_salary_next.reset_index(drop=True)
print(data_salary_next.shape)
data_salary_next.head()

(4765, 33)


,No,Name,Year,Team,Position,G,PA,AB,R,H,...,SF,AVG,OBP,SLG,OPS,wOBA,wRC+,WAR,WPA,Salary
0,,가득염,2007,S,P,4,0,0,0,0,...,0,,,,,,,0.00,,"8,000"
1,117,가득염,2008,S,P,3,1,1,0,1,...,0,1.000,1.000,1.000,2.000,.955,529.9,0.05,,"11,000"
2,,가득염,2010,S,P,1,0,0,0,0,...,0,,,,,,,0.00,,"9,500"
3,106,강경학,2014,한,SS,41,103,86,11,19,...,0,.221,.337,.349,.686,.316,65.6,0.21,-0.11,"2,400"
4,123,강경학,2015,한,SS,120,376,311,50,80,...,3,.257,.348,.325,.673,.304,66.4,0.11,-0.32,"6,500"


In [81]:
data_salary_next["NextSalary"] = "0"

for i in range(len(data_salary_next) - 1):
    currentYear = data_salary_next.loc[i]
    nextYear = data_salary_next.loc[i+1]
    
    if str(currentYear["Name"]) == str(nextYear["Name"]) and int(currentYear["Year"]) == (int(nextYear["Year"]) - 1):
        data_salary_next.loc[i]["NextSalary"] = nextYear["Salary"]
    else:
        data_salary_next = data_salary_next.drop(i, 0)

In [83]:
data_salary_next.to_csv('KBO_1997_2020_Salary_Next_EN.csv', encoding='cp949', index=False)
data_salary_next.head()

,No,Name,Year,Team,Position,G,PA,AB,R,H,...,AVG,OBP,SLG,OPS,wOBA,wRC+,WAR,WPA,Salary,NextSalary
0,,가득염,2007,S,P,4,0,0,0,0,...,,,,,,,0.00,,"8,000","11,000"
3,106,강경학,2014,한,SS,41,103,86,11,19,...,.221,.337,.349,.686,.316,65.6,0.21,-0.11,"2,400","6,500"
4,123,강경학,2015,한,SS,120,376,311,50,80,...,.257,.348,.325,.673,.304,66.4,0.11,-0.32,"6,500","6,500"
5,370,강경학,2016,한,SS,46,116,101,16,16,...,.158,.232,.257,.490,.223,10.5,-0.69,-1.10,"6,500","6,200"
6,315,강경학,2017,한,2B,59,93,84,17,18,...,.214,.290,.262,.552,.264,44.1,-0.18,-0.81,"6,200","5,800"


----

# 개명 선수 크롤링

KBO 홈페이지는 jQuery를 이용해 화면 내용을 변경하는 경우가 있다. 이를 고려해야 한다.

In [ ]:
# 필요한 라이브러리 import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
import time    

# setup Driver | Chrome: 크롬 드라이버를 사용하는 driver 생성
driver = webdriver.Chrome('./chromedriver')  # Chrome driver 경로
driver.implicitly_wait(10)

In [ ]:
# select option 값을 변경하고 버튼을 누르는 경우를 고려해야 함
driver.get('https://www.koreabaseball.com/Player/Trade.aspx')
time.sleep(2)

selYear = Select(driver.find_element(By.ID, "selYear"))
selSection = Select(driver.find_element(By.ID, "selSection"))
btnSearch = driver.find_element(By.ID, "btnSearch")

result = []
for y in range(2017, 2021):
    count = 0
    selYear.select_by_value(str(y))
    selSection.select_by_visible_text('개명')
    btnSearch.click()
    time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    tr = soup.select('table#tblTradeList > tbody > tr')
    
    for r in tr:
        row = []
        td = r.find_all('td')
        row.append(td[0].text.strip()[0:4]) # 연도
        row.append(td[2].text.strip()) # 팀
        row.append(td[3].text.strip()[:td[3].text.strip().find('(')])
        if td[4].text.strip().find('(') > -1:
            row.append(td[4].text.strip()[td[4].text.strip().find(':') + 1:td[4].text.strip().find('(')])
        else:
            row.append(td[4].text.strip()[td[4].text.strip().find(':') + 1:])
        
        if len(row) > 0:
            count += 1
            result.append(row)
        
    print(y, '년: ', count, sep='')

rename = pd.DataFrame(result)

In [ ]:
rename.head()

In [ ]:
rename.columns = ['시즌', '팀', '개명 전', '개명 후']
rename.to_csv('KBO_2017_2020_Rename_KR.csv', encoding='cp949', index=False)
rename.head()